# Imports

In [39]:
from numpy.random import seed
seed(888)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(404)

In [40]:
import os
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# Constants

In [41]:
x_train_path='handwriting/digit_xtrain.csv'
y_train_path='handwriting/digit_ytrain.csv'
x_test_path='handwriting/digit_xtest.csv'
y_test_path='handwriting/digit_ytest.csv'
nr_classes=10
valid_size=10000
total_inputs=28*28*1 # image height * width * channels

# Get the Data

In [42]:
%%time
x_train_all=np.loadtxt(x_train_path, delimiter=',', dtype=int)

Wall time: 18.1 s


In [43]:
%%time
x_test=np.loadtxt(x_test_path, delimiter=',', dtype=int)
y_train_all=np.loadtxt(y_train_path, delimiter=',', dtype=int)
y_test=np.loadtxt(y_test_path, delimiter=',', dtype=int)

Wall time: 3.09 s


# Preprocessing Data

## Rescale the data to from 255 to 1.0

In [44]:
x_train_all, x_test = x_train_all/255.0 , x_test/ 255.0

## Convert our Y set lables to One Hot Encodeing with numpy eye()

In [45]:
y_train_all= np.eye(nr_classes)[y_train_all]

In [46]:
y_test=np.eye(nr_classes)[y_test]

In [47]:
y_train_all.shape

(60000, 10)

### make validation sets from trainig datasets 

In [48]:
x_val=x_train_all[:valid_size]
y_val=y_train_all[:valid_size]

In [49]:
x_train=x_train_all[valid_size:]
y_train=y_train_all[valid_size:]

In [50]:
x_train.shape

(50000, 784)

In [51]:
x_val.shape

(10000, 784)

# Setting and Creating Tensors

In [52]:
x=tf.compat.v1.placeholder(tf.float32, shape=(None, total_inputs))
y=tf.compat.v1.placeholder(tf.float32, shape=(None, nr_classes))


## Neural Network Architcture

### defining our Hyperparams

In [53]:
nr_epochs= 5
learning_rate =1e-4
n_hidden1=512
n_hidden2=64

## Initialization 

### first Layer

In [54]:
init_w1=tf.compat.v1.truncated_normal(shape=[total_inputs, n_hidden1], stddev=0.1, seed=42)
w1= tf.Variable(init_w1)
init_b1=tf.constant(value=0.0, shape=[n_hidden1])
b1=tf.Variable(init_b1)

In [55]:
layer1_in = tf.matmul(x, w1)+ b1
layer1_out= tf.nn.relu(layer1_in)

### second Layer

In [56]:
init_w2=tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
w2= tf.Variable(init_w2)
init_b2=tf.constant(value=0.0, shape=[n_hidden2])
b2=tf.Variable(init_b2)

In [57]:
layer2_in = tf.matmul(layer1_out, w2)+ b2
layer2_out= tf.nn.relu(layer2_in)

### output Layer

In [58]:
init_w3=tf.compat.v1.truncated_normal(shape=[n_hidden2, nr_classes], stddev=0.1, seed=42)
w3= tf.Variable(init_w3)
init_b3=tf.constant(value=0.0, shape=[nr_classes])
b3=tf.Variable(init_b3)

In [59]:
layer3_in = tf.matmul(layer2_out, w3)+ b3
output= tf.nn.softmax(layer3_in)

# Setup Loss, Optimizer, Metrics

### Loss

In [60]:
loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=output))

### Optimizer

In [61]:
optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate)
train_step=optimizer.minimize(loss)

### Accuracy Metric

In [71]:
correct_pred=tf.equal(tf.argmax(output, axis=1),tf.argmax(y,axis=1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))
accuracy

<tf.Tensor 'Mean_5:0' shape=() dtype=float32>

# Run Session

In [63]:
sess=tf.compat.v1.Session()

In [64]:
# initialisation all the Variables
init=tf.compat.v1.global_variables_initializer()
sess.run(init)

## Batching the Data

In [65]:
size_of_batch=1000
num_examples=y_train.shape[0]
nr_iterations=int(num_examples/size_of_batch)
index_in_epoch=0

In [66]:
nr_iterations

50

### Build Function to help batching the Data:

In [72]:
def next_batch(batch_size, data, labels):
    global num_examples
    global index_in_epoch
    # if we reach the end of the Dataset
    start=index_in_epoch
    index_in_epoch += batch_size
    if index_in_epoch > num_examples:
        start=0
        index_in_epoch=batch_size

    end=index_in_epoch

    return data[start:end] , labels[start:end]

# Trainig Loop:

In [83]:
for epoch in range(50):
    for i in range(nr_iterations):
        batch_x, batch_y= next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary= {x:batch_x, y:batch_y}
        sess.run(train_step, feed_dict=feed_dictionary)
        batch_accuracy=sess.run(fetches=[accuracy], feed_dict=feed_dictionary)
    print(f'Epoch {epoch} \t| Trainig Accuracy {batch_accuracy}')
print("Training Done!")

Epoch 0 	| Trainig Accuracy [0.992]
Epoch 1 	| Trainig Accuracy [0.992]
Epoch 2 	| Trainig Accuracy [0.992]
Epoch 3 	| Trainig Accuracy [0.992]
Epoch 4 	| Trainig Accuracy [0.992]
Epoch 5 	| Trainig Accuracy [0.992]
Epoch 6 	| Trainig Accuracy [0.992]
Epoch 7 	| Trainig Accuracy [0.992]
Epoch 8 	| Trainig Accuracy [0.992]
Epoch 9 	| Trainig Accuracy [0.992]
Epoch 10 	| Trainig Accuracy [0.992]
Epoch 11 	| Trainig Accuracy [0.992]
Epoch 12 	| Trainig Accuracy [0.992]
Epoch 13 	| Trainig Accuracy [0.992]
Epoch 14 	| Trainig Accuracy [0.992]
Epoch 15 	| Trainig Accuracy [0.992]
Epoch 16 	| Trainig Accuracy [0.992]
Epoch 17 	| Trainig Accuracy [0.992]
Epoch 18 	| Trainig Accuracy [0.992]
Epoch 19 	| Trainig Accuracy [0.992]
Epoch 20 	| Trainig Accuracy [0.992]
Epoch 21 	| Trainig Accuracy [0.992]
Epoch 22 	| Trainig Accuracy [0.992]
Epoch 23 	| Trainig Accuracy [0.992]
Epoch 24 	| Trainig Accuracy [0.992]
Epoch 25 	| Trainig Accuracy [0.992]
Epoch 26 	| Trainig Accuracy [0.992]
Epoch 27 	|